In [1]:
import pickle
import os 
import cv2
import glob
import matplotlib.pyplot as plt

# select one of the multiple GPU cards on the server.
os.environ["CUDA_VISIBLE_DEVICES"]="0" # set GPU if multiple present




In [2]:
# the pkl directory contins the pkl files that hold a DeepLearning structure that defines the DL model
pklDir = input('Enter pkl path or use default\n') or '/media/amw/TOSHIBA EXT/alerding/models/pkl/'
# the model directory will receive the trained model
modelDir = input('Enter model output directory or default\n') or '/media/amw/TOSHIBA EXT/alerding/models/gpuModels/'
    
loadweights = None

# modelname is the name of hte pkl file without its pkl  extension.
# a pkl file holds a data structure or object. IN this case, it is expected to hold a DeepLearning object that was saved
# using File>Save DL Object... in mianalyzer. 
modelname = input("Enter modelname without extension or use default") or "unetDensnet201_2023-01-24_08-17-37"


numclasses = 4

# trainingdata is a directory that holds the annotated images. The labels must be in trainingdata/Segmentation_labels/
# and they must be npz files.
trainingdata = '/media/amw/TOSHIBA EXT/alerding/annotated/notCroppedLabels2/resized/'
validationdata = None

print("model: ", modelname)


model:  unetDensnet201_2023-01-24_08-17-37


In [3]:
print('load settings')
#filehandler = open(modelname + '.pkl', 'rb')
filehandler = open(pklDir + modelname + '.pkl', 'rb')



dl = pickle.load(filehandler)

from pprint import pprint

pprint(vars(dl))


load settings
{'ImageScaleFactor': 0.5,
 'Mode': <dl.method.segmentation.Segmentation object at 0x7fa35a82fb50>,
 'Model': None,
 'TrainInMemory': True,
 'augmentation': <dl.training.augment.ImageAugment object at 0x7fa35a82f150>,
 'batch_size': 8,
 'borderremoval': 112,
 'data': <dl.data.imagedata.ImageData object at 0x7fa35a835e10>,
 'dextr': <dl.machine_learning.dextr.dextr_segmentation.DEXTR_Segmentation object at 0x7fa35a78e890>,
 'epochs': 50,
 'grabcut': <dl.machine_learning.grabcut_segmentation.GrabCutSegmentation object at 0x7fa35a835b10>,
 'hed': None,
 'interrupted': False,
 'learning_rate': 0.004,
 'lrschedule': <dl.training.lrschedule.LearningRateSchedule object at 0x7fa35a833490>,
 'mixed_precision': False,
 'observer': None,
 'observers': [],
 'od_kernel_size': 3,
 'od_peak_val': 125,
 'optimizer': <dl.optimizer.optimizer.Optimizer object at 0x7fa35a833650>,
 'preprocess': <dlPreprocess.scale: 0>,
 'record': <dl.training.training_record.TrainingRecording object at 0x7fa3

In [4]:
print(os.getcwd())

from dl.DeepLearning import *
def createDL( epochs=100, scaleFactor=0.5, learning_rate = 0.001):
    dl = DeepLearning()
    dl.Epochs = epochs
    dl.ImageScaleFactor = scaleFactor
#    dl.Mode = Segmentation()
    dl.learning_rate = learning_rate
    dl.Mode.architecture = 'UNet'
    dl.Mode.backbone = 'resnet152'
    return dl




    
    




/home/amw/git/miaHeadless/mia


In [ ]:
dl = createDL( epochs=5, scaleFactor=0.5, learning_rate = 0.002)
numClasses =4

print('init model')
dl.initModel(numclasses)

dl.Model = dl.Mode.getModel(numClasses, 3)

print('load model')
if loadweights is not None:
    dl.Model.load_weights(loadweights)

print(dl.initialized)

In [7]:
        
# print('load settings')
#    filehandler = open(modelname + '.pkl', 'rb')
# filehandler = open(pklDir + modelname + '.pkl', 'rb')

# dl = pickle.load(filehandler)

# the dl parameters can be modified, either through methods in DeepLearning.py or by modifying public variables.
# here I can set teh number of epochs, so that a new pkl file does not need to be created when I change the number of epochs
# dl.epochs=80

# it shold be possibleto define the entire structure wihtout a pkl file, but I have not yet figured out how to set or read 
# the architecture and backbone.

dl = createDL( epochs=5, scaleFactor=0.5, learning_rate = 0.002)
numClasses =4

print('init model')
dl.initModel(numclasses)
print('load model')
if loadweights is not None:
    dl.Model.load_weights(loadweights)
        
if dl.initialized:
    print('start training')
    dl.initData_StartTraining(trainingdata, validationdata)
    print('training finished')
else:
    print('could not initialize model')
            
    
print('saving weights')
dl.Model.save_weights(modelDir+modelname + '.h5')
print('saving training data')
dl.saveTrainingRecord(modelDir+modelname +'.csv')
#   print('saving model')
#   dl.Model.save(modelDir+'full')
    



load model
could not initialize model
saving weights


2023-06-06 14:37:41.488367: I tensorflow/stream_executor/cuda/cuda_driver.cc:732] failed to allocate 15.48M (16233216 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-06-06 14:37:41.488622: I tensorflow/stream_executor/cuda/cuda_driver.cc:732] failed to allocate 15.48M (16233216 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-06-06 14:37:41.488652: W tensorflow/core/common_runtime/bfc_allocator.cc:457] Allocator (GPU_0_bfc) ran out of memory trying to allocate 144.0KiB (rounded to 147456)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-06-06 14:37:41.488705: I tensorflow/core/common_runtime/bfc_allocator.cc:1004] BFCAllocator dump for GPU_0_bfc
2023-06-06 14:37:41.488714: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (256): 	Total Chunks: 134, Chunks in use: 13

AttributeError: 'NoneType' object has no attribute 'save_weights'

In [4]:
%matplotlib inline

In [ ]:
# This cell does a prediction on a directory of jpg images (could be any type, but that is what I have).
# predictionFolder contains the images which ill be predicted, and a subdirectory with eh modelname will be created
# to hold the predicted segmentation.

predictionFolder = input('Enter folder for prediction or use default\n')  or '/home/dsi/aryeh/data/plants/Harvest8Orange5_7Oct17/'
outputPath = predictionFolder + modelname+'_ep'+str(dl.epochs)+'/'
try:
    os.mkdir(outputPath)
except FileExistsError:
    pass
files = glob.glob(predictionFolder + '*.jpg')
print(len(files))

# If we do not want to process the entire folder, we can limit the number of processed image to count. 
count = 0
for i in files:
    print(i)
    img = cv2.imread(i)
    prediction = dl.Mode.PredictImage(img)
    plt.figure(count)
    plt.imshow(prediction)
    plt.show()
    cv2.imwrite(outputPath+'segmented_'+os.path.basename(i).replace('jpg', 'png'), prediction)
    count += 1
 #   if count > 1:
 #      break
    



In [1]:
print(dl.method.pixelbasedprediction.architecture)

NameError: name 'dl' is not defined

In [26]:
# I was checking why repeated runs of the prediction produced different reauslts. One possibility is that
# I need to reset the DL object between runs. But I am not sure about this.
print(dl.Model)
dl.cleanMemory()

0
